Importing the modules os and pandas and the classes ML_GRB and RND_FOREST from the ML_GRB.py file.
I will use "os" to take the path of the working folder for the data reading part, while "pandas" to manage the DataFrame.

The class ML_GRB is the main class of the project, while RND_FOREST is the class for the specific model to use in the main class "ML_GRB". The idea is to create other model classes like "RND_FOREST" using similiar structure betwen the model class and the main class (see "ML_GRB.py" for more details).

In [4]:
import os
import pandas as pd
from ML_GRB import ML_GRB, RND_FOREST

Now I intitialize the main class and save it in the variable "obj". Then I use the function 
DataReading (see functions.py) to read the data from the folder "data". This function takes in 
input the working folder path and saves the data as istances in the class

In [5]:
# dir_path = os.path.dirname(os.path.abspath(__file__))   # directory path (This doesn't 
#                                                           work in python notebook, but it
#                                                           works very well in python files, 
#                                                           so here I use the line below, and
#                                                           one can change it if necessary.)
dir_path = ''       # directory path

obj = ML_GRB()      # initializing the main class ---> see ML_GRB.py
obj.DataReading(dir_path)       # reading the data ---> see functions.py

Here I build the dataset to work with.
First I take the DataFrames of spectral and energetic parameters and I save they in two variables.
Then I take just the features useful for the model (for now I take the features below for a test but the code should works fine even with all the necessary variables of the dataset).
After that I concatenate the two DataFrame to a single DataFrame tha will works as our 
dataset for the model.
Here I choose to take only the data fitted with the CPL model just for a test, but honestly 
the implication of this are a bit confusing for me. This could be one of the theme for a possible 
next meeting.

Finally one can filter the dataset again changing the features list.
I've done this because in my mind the first filter is useful to select the features usable in general, 
while the last filter is to select the specific features to use in the model.

In [9]:
# Taking the dataframes from the class
spec_par = obj.spectral_data
energ_par = obj.energetic_data

# Defining the features to use in the model from the two dataframes
filter_ener = ['z', 'S', 'Tp64', 'Fp64', 'Eiso','Liso']
filter_spec = ['tstart', 'DeltaT', 'alpha', 'Ep', 'F']

spec_par_reset = spec_par.reset_index(drop=True)  # Reset index and drop the old index column

# Concatenating the two dataframe to form one and filtering
df1 = energ_par[energ_par['ID'] != 'GRB080413B'][filter_ener]   # Exluding GRB080413B becuase 'SMOD' = 'PL'
df2 = spec_par_reset[(spec_par['SMod'] == 'CPL') & (spec_par['SType'] == 'i')][filter_spec]    # Using the CPL SMod
df_temp = pd.concat([df1, df2], axis=0)     # Concatenating
df = df_temp.apply(lambda x: pd.Series(x.dropna().values))      # Dropping the Nan values

# Selecting the features to use (one can change directly from here if he wants)
features = ['z', 'S', 'Tp64', 'Fp64', 'Eiso','Liso', 
          'tstart', 'DeltaT', 'alpha', 'Ep', 'F']
dataset = df[features]

In the next cell I initialize the model class and save it in the variable "model", then 
I set the model in the main class (see ML_GRB.py). 
After that I train the model with the function Run (see functions.py)

In [10]:
# Inizialising the model class ---> see ML_GRB.py
model = RND_FOREST()
obj.SetModel(model)     # Setting the model in the main class

# Fitting the model ---> see functions.py
obj.model.Run(dataset, train_size = 0.66, random_state = 42, n_estimators = 1000)

In the last cell I print the results: 
- the values of redshift from the test data compared with the values of redshift predicted by the model
- the score on the test data
- the score on the train data

In the next future I want to implement some consinstent tools to evaluate the model.
For now I have printed only simple results that doesn't has much estimation values, just
to test the code.

In [11]:
# Printing the results
test_score = obj.model.forest.score(obj.model.X_test, obj.model.y_test)
train_score = obj.model.forest.score(obj.model.X_train, obj.model.y_train)

for a, b in zip(obj.model.y_test, obj.model.y_pred):
    print('z real: {:1.3f}\t\tz predicted: {:1.3f}'.format(a[0], b))
    
print('The test score is : {:.3f} while the train score is: {:.3f}'.format(test_score, train_score))

z real: 2.770		z predicted: 2.908
z real: 0.360		z predicted: 0.361
z real: 0.360		z predicted: 0.363
z real: 0.540		z predicted: 0.538
z real: 3.600		z predicted: 3.692
z real: 2.770		z predicted: 2.811
z real: 2.690		z predicted: 2.689
z real: 0.760		z predicted: 0.718
z real: 4.350		z predicted: 3.907
z real: 3.000		z predicted: 3.135
z real: 1.410		z predicted: 1.417
z real: 2.030		z predicted: 2.081
z real: 2.300		z predicted: 2.296
z real: 1.020		z predicted: 1.021
z real: 0.690		z predicted: 0.703
z real: 1.330		z predicted: 1.297
z real: 0.590		z predicted: 0.532
z real: 0.690		z predicted: 0.703
z real: 0.820		z predicted: 0.824
z real: 1.320		z predicted: 1.304
z real: 2.610		z predicted: 2.524
z real: 2.400		z predicted: 2.457
z real: 2.900		z predicted: 2.954
z real: 0.890		z predicted: 0.889
z real: 1.030		z predicted: 1.034
z real: 1.720		z predicted: 1.771
z real: 2.490		z predicted: 2.470
z real: 1.610		z predicted: 1.599
z real: 0.110		z predicted: 0.180
z real: 2.200	